# Actividad de clase: Ajuste de hiperparámetros en Python

En esta actividad se exploran distintas metodologías de ajuste de hiperparámetros

Se usará el dataset iris de sklearn para tratar de predecir el tipo de flor con base en la información de cuatro variables de entrada.

Supongamos que se usó un método de clasificación SVM. ¿Cuáles hiperparámetros funcionarán mejor?

En la siguiente tabla se muestran algunos de los hiperparámetros del modelo SVM para clasificación.

| Parámetro | Valores posibles |
| :-: | :-: |
| Kernel | 'rbf','linear',poly |
| C | Integer |
| Gamma | Float |

¿Cuál combinación de hiperparámetros permite conseguir una precisión mayor en el modelo?

Cargado de librerías

In [2]:
import pandas as pd
import numpy as np


Cargado de datos

In [1]:
# Importación de los datos de sklearn
from sklearn import svm, datasets
# Asignación de los datos a un objeto
iris = datasets.load_iris()


Asignación de datos a un objeto de nombre df

In [3]:
df = pd.DataFrame(iris.data, columns=iris.feature_names)


Vista preliminar del df

In [4]:
df.head()


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


Creación de una columna 'flower' y asignación del tipo de flor

In [5]:
# Creación de columna
df['flower'] = iris.target
# Visualización
df.head()


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [6]:
# Cambiar los números por los nombres
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])
# Visualización
df[47:52]


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor


División de los datos en conjuntos de entrenamiento y test

In [9]:
# Importación de la función
from sklearn.model_selection import train_test_split
# Partición de los datos
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.3)


Implementación del modelo SVM para clasificación y cálculo de la precisión dada la elección de parámetros

In [10]:
# Creación del objeto
model = svm.SVC(kernel='rbf', C=30, gamma='auto')
# Ajuste
model.fit(X_train, y_train)
# Cálculo del score del modelo
model.score(X_test, y_test)


0.9555555555555556

# Implementación de k-fold Cross Validation

In [11]:
from sklearn.model_selection import cross_val_score

Ensayo 1: kernel='linear', C=10, gamma=auto

In [12]:
cross_val_score(svm.SVC(kernel='linear', C=10, gamma='auto'),
                iris.data, iris.target, cv=5)


array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

Ensayo 2: kernel='rbf', C=30, gamma=auto

In [13]:
cross_val_score(svm.SVC(kernel='rbf', C=30, gamma='auto'),
                iris.data, iris.target, cv=5)


array([0.96666667, 1.        , 0.9       , 0.93333333, 1.        ])

# Implementación de Grid Search CV

In [14]:
# Importación del método
from sklearn.model_selection import GridSearchCV
# Creación del objeto
clf = GridSearchCV(svm.SVC(gamma='auto'), {
                   'C': [1, 10, 30],
                   'kernel': ['linear', 'rbf']},
                   cv=5,
                   return_train_score=False
                  )
# Ajuste
clf.fit(iris.data, iris.target)
# Visualización de resultados
clf.cv_results_

{'mean_fit_time': array([0.00231137, 0.00147514, 0.00084004, 0.00084605, 0.00066338,
        0.00098534]),
 'std_fit_time': array([0.00087777, 0.00037553, 0.00020981, 0.00015562, 0.00011   ,
        0.00025489]),
 'mean_score_time': array([0.00092564, 0.00090575, 0.00033798, 0.00036922, 0.00029602,
        0.00048666]),
 'std_score_time': array([4.28862701e-04, 1.52867181e-04, 3.87940364e-05, 5.61248881e-05,
        4.27222145e-05, 7.38733571e-05]),
 'param_C': masked_array(data=[1, 1, 10, 10, 30, 30],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['linear', 'rbf', 'linear', 'rbf', 'linear', 'rbf'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'linear'},
  {'C': 1, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 30, 'kernel': 'linear'},
  {'C':

In [16]:
# Asignar los resultados a un dataframe
results = pd.DataFrame(clf.cv_results_)
# Ver los resultados
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002311,0.000878,0.000926,0.000429,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.001475,0.000376,0.000906,0.000153,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.000840,0.000210,0.000338,0.000039,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
3,0.000846,0.000156,0.000369,0.000056,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
4,0.000663,0.000110,0.000296,0.000043,30,linear,"{'C': 30, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.900000,1.0,0.960000,0.048990,5


In [23]:
# Tomar solo algunas columnas
results[['param_C', 'param_kernel', 'mean_test_score']
        ].sort_values(by='mean_test_score', ascending=False)


,param_C,param_kernel,mean_test_score
0,1,linear,0.980000
1,1,rbf,0.980000
3,10,rbf,0.980000
2,10,linear,0.973333
4,30,linear,0.960000
5,30,rbf,0.960000


In [24]:
# Ver el mejor resultado obtenido
clf.best_score_

0.9800000000000001

In [25]:
# Ver los parámetros que logran el mejor resultado
clf.best_params_

{'C': 1, 'kernel': 'linear'}

# Implementación de Randomized Size CV

In [27]:
# Importación del método
from sklearn.model_selection import RandomizedSearchCV
# Creación del objeto
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
                        'C': [1, 10, 30],
                        'kernel': ['linear', 'rbf']},
                        cv=5,
                        return_train_score=False,
                        n_iter=2
                        )


In [28]:
# Ajuste
rs.fit(iris.data, iris.target)
# Visualización de resultados
pd.DataFrame(rs.cv_results_)[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,30,rbf,0.960000
1,10,linear,0.973333
